# Grand canonical ensemble transition-matrix Monte Carlo

In this example, flat histogram methods are employed for a small macrostate range from 0 to 5 particles.
To begin, the system is initialized with the minimum number of particles by setting Metropolis acceptance criteria with favorable conditions for adding particles.
The Metropolis criteria are then replaced with the flat histogram criteria.
At this point, typical analysis from the previous tutorials are added.
In addition, we also add checkpoint files, criteria status, and average energy of a given macrostate.
Finally, the simulation is run until the requested number of iterations of the flat histogram algorithm are complete.

A small macrostate range allows the simulation to run quickly with good sampling, and thus it is an ideal starting point to test the simulations. To begin, read the previous SRSW values from file for comparison.

In [1]:
import pandas as pd
import feasst as fst

srsw_file = fst.install_dir() + "/plugin/flat_histogram/test/data/stat150.csv"
ln_prob_srsw = fst.LnProbability(pd.read_csv(srsw_file)["lnPI"].values[:6])
ln_prob_srsw.normalize() # normalize to account for a smaller macrostate range
df = pd.DataFrame(data=ln_prob_srsw.values(), columns={"ln_prob_srsw"})
df['ln_prob_srsw_std'] = 0.04
df['u_srsw'] = pd.read_csv(srsw_file)["energy"]
df['u_srsw_std'] = pd.read_csv(srsw_file)["energystd"]
df

,ln_prob_srsw,ln_prob_srsw_std,u_srsw,u_srsw_std
0,-18.707570,0.04,-2.312265e-10,6.689238e-10
1,-14.037373,0.04,-6.057402e-04,6.709198e-10
2,-10.050312,0.04,-3.057422e-02,9.649147e-06
3,-6.458921,0.04,-8.992832e-02,1.387472e-04
4,-3.145637,0.04,-1.784571e-01,3.315245e-05
5,-0.045677,0.04,-2.961920e-01,1.348791e-05


In [2]:
import unittest
import pyfeasst

def run_sample_lj_tm_mc(checkpoint_file_name):
    monte_carlo = fst.MonteCarlo()
    monte_carlo.add(fst.MakeConfiguration(fst.args({"cubic_box_length": "8",
                                       "particle_type0": fst.install_dir() + "/forcefield/lj.fstprt"})))
    monte_carlo.add(fst.MakePotential(fst.MakeLennardJones()))
    monte_carlo.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
    monte_carlo.set(fst.MakeThermoParams(fst.args({"beta": str(1./1.5), "chemical_potential": "-2.352321"})))
    monte_carlo.set(fst.MakeFlatHistogram(
        fst.MakeMacrostateNumParticles(fst.Histogram(fst.args({"width": "1", "min": "0", "max": "5"}))),
        fst.MakeTransitionMatrix(fst.args({"min_sweeps": "50"}))))
    monte_carlo.add(fst.MakeTrialTranslate(fst.args({"weight": "0.25", "tunable_param": "1."})))
    monte_carlo.add(fst.MakeTrialTransfer(fst.args({"weight": "1", "particle_type": "0"})))
    monte_carlo.add(fst.MakeLogAndMovie(fst.args({"trials_per": str(1e5), "file_name": "lj"})))
    monte_carlo.add(fst.MakeCheckEnergyAndTune(fst.args({"trials_per": str(1e5), "tolerance": str(1e-8)})))
    monte_carlo.add(fst.MakeCriteriaUpdater(fst.args({"trials_per": str(1e5)})))
    monte_carlo.add(fst.MakeCriteriaWriter(fst.args({"trials_per": str(1e5), "file_name": "lj_fh.txt"})))
    monte_carlo.add(fst.MakeEnergy(fst.args({"file_name": "lj_en.txt", "trials_per_update": "1",
        "trials_per_write": str(1e5), "multistate": "true"})))
    monte_carlo.set(fst.MakeCheckpoint(fst.args({"file_name": checkpoint_file_name, "num_hours": "0.0001"})))
    monte_carlo.run_until_complete()

class TestFlatHistogramLJ(unittest.TestCase):
    """Test flat histogram grand canonical ensemble Monte Carlo simulations"""
    def test_serial_5max(self):
        """Compare the free energies and potential energies with the NIST SRSW
        https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
        https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
        """
        # To emulate post-processing, obtain monte_carlo from checkpoint file
        checkpoint_file_name='checkpoint.txt'
        run_sample_lj_tm_mc(checkpoint_file_name)
        monte_carlo = fst.MonteCarlo().deserialize(pyfeasst.read_checkpoint(checkpoint_file_name))

        # To compare with previous values, make a deep copy of the FlatHistogram derived class
        criteria = fst.FlatHistogram(monte_carlo.criteria())
        print('lnpi energy')
        for macro in range(criteria.num_states()):
            self.assertAlmostEqual(
                df["ln_prob_srsw"][macro],
                criteria.bias().ln_prob().value(macro),
                delta=df["ln_prob_srsw_std"][macro])
            energy_analyzer = monte_carlo.analyze(monte_carlo.num_analyzers() - 1)
            energy_accumulator = energy_analyzer.analyze(macro).accumulator()
            stdev = (df["u_srsw_std"][macro]**2 + energy_accumulator.block_stdev()**2)**(1./2.)
            #print(criteria.bias().ln_prob().value(macro), energy_accumulator.average())
            self.assertAlmostEqual(
                df["u_srsw"][macro],
                energy_accumulator.average(),
                delta=5*stdev)

In [3]:
# Note: any line starting with % is only to be used with ipynb
%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_serial_5max (__main__.TestFlatHistogramLJ)
Compare the free energies and potential energies with the NIST SRSW ... 

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11.4 µs
lnpi energy


ok

----------------------------------------------------------------------
Ran 1 test in 11.283s

OK


A number of files should also have been created.
If the flat histogram method is sampling perfectly, the simulation performs a random walk along the macrostate.
For larger ranges of macrostates, or for more difficult sampling cases, monitoring the macrostate can help you determine what conditions are preventing convergence.
For example, a plot of the macrostate as a function of the number of attempts may look like the following:


In [4]:
pd.read_csv("lj.txt", header=0).dropna(axis='columns')

,volume,p0,beta,state,energy,attempt,TrialTranslate,tunable,TrialAdd,TrialRemove
0,512,5,0.666667,5,-4.358780e-01,100000,0.943786,1.00000,0.044366,0.044385
1,512,3,0.666667,3,-1.127030e-02,200000,0.960806,1.05000,0.694640,0.708879
2,512,0,0.666667,0,-2.802770e-15,300000,0.971389,1.10250,0.807342,0.967410
3,512,2,0.666667,2,-2.422960e-03,400000,0.971608,1.15763,0.812283,0.972804
4,512,0,0.666667,0,2.021060e-15,500000,0.970708,1.21551,0.814323,0.972938
5,512,1,0.666667,1,-6.057400e-04,600000,0.967800,1.27628,0.815516,0.975552
6,512,1,0.666667,1,-6.057400e-04,700000,0.970213,1.34010,0.816728,0.973423
7,512,4,0.666667,4,-9.691840e-03,800000,0.965229,1.40710,0.808667,0.971636
8,512,0,0.666667,0,-9.192950e-16,900000,0.965878,1.47746,0.811482,0.972543
9,512,1,0.666667,1,-6.057400e-04,1000000,0.965261,1.55133,0.814702,0.974363


Note that states are index integer values starting from 0 (e.g., 0, 1, 2, ..., criteria.num_states() - 1)
The state and macrostate happen to be the same when the minimum macrostate is 0, and the macrostate is the integer number of particles.
But if the minimum macrostate was 1, then state 0 would correspond to macrostate 1.0.
Obtain an arbitrary macrostate value from the state as follows.

In [5]:
monte_carlo = fst.MonteCarlo().deserialize(pyfeasst.read_checkpoint("checkpoint.txt"))
criteria = fst.FlatHistogram(monte_carlo.criteria())
print('state macrostate')
for state in range(criteria.num_states()):
    print(state, criteria.macrostate().value(state))

state macrostate
0 0.0
1 1.0
2 2.0
3 3.0
4 4.0
5 5.0


Many simulation parameters may be obtained from the checkpoint file to automate your analysis.

In [6]:
print('volume', monte_carlo.configuration().domain().volume())
print('beta', monte_carlo.thermo_params().beta())
print('beta_mu', monte_carlo.thermo_params().beta_mu())
print('macro_min', criteria.macrostate().value(0))  # monte_carlo.critera() doesn't know macrostate. Use copy of derived class
print('macro_max', criteria.macrostate().value(criteria.num_states() - 1))
print('macro_max', criteria.macrostate().histogram().center_of_last_bin())

volume 512.0
beta 0.6666666666666666
beta_mu -1.5682139999999998
macro_min 0.0
macro_max 5.0
macro_max 5.0


The energy of each macrostate may also be compared with the published values in the NIST SRSW.

In [7]:
en = pd.read_csv("lj_en.txt").rename(columns={"average": "u", "block_stdev": "u_std"})
pd.concat([pd.DataFrame(en[["u", "u_std"]]), df[["u_srsw", "u_srsw_std"]]], axis=1)

,u,u_std,u_srsw,u_srsw_std
0,8.362240e-17,3.496062e-16,-2.312265e-10,6.689238e-10
1,-6.057400e-04,2.174108e-12,-6.057402e-04,6.709198e-10
2,-3.022146e-02,1.914936e-04,-3.057422e-02,9.649147e-06
3,-8.944287e-02,6.205232e-04,-8.992832e-02,1.387472e-04
4,-1.776325e-01,1.183672e-03,-1.784571e-01,3.315245e-05
5,-2.975943e-01,1.263983e-03,-2.961920e-01,1.348791e-05


You may also compare the natural logarithm of the macrostate probability

In [8]:
pd.concat([df["ln_prob_srsw"], pd.read_csv("lj_fh.txt", header=1)['ln_prob']], axis=1)

,ln_prob_srsw,ln_prob
0,-18.707570,-18.708853
1,-14.037373,-14.036188
2,-10.050312,-10.047908
3,-6.458921,-6.456178
4,-3.145637,-3.143491
5,-0.045677,-0.045779


The macrostate probability distribution depends upon the choice of the chemical potential, but can be reweighted to different chemical potentials.
The collection matrix may be accessed directly as:

In [9]:
tm = fst.TransitionMatrix(criteria.bias())
print(tm.collection().matrix())

((0.0, 490658.0, 327110.0), (3069.581954269688, 491835.8487120219, 326024.5693312088), (6104.062418983776, 495331.5028214753, 328145.43476144026), (9169.333423416996, 501944.48620484327, 330625.1803712205), (12160.1071825707, 506374.6668414039, 331579.22597497044), (16547.69186126913, 558631.3134046715, 364688.9947327478))


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!